In [14]:
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm

In [15]:
th1 = 0.95 #confidence threshold
th2 = 10 #max L-R discrepancy in x coordinate

root_folder = '/data/LiftFly3D/prism/'
scorer_bottom = 'DLC_resnet50_jointTrackingDec13shuffle1_200000'
scorer_side = 'DLC_resnet50_sideJointTrackingDec17shuffle1_200000'

#joints whose confidence to consider
joints = ['tarsus tip front L', 'tarsus tip mid L', 'tarsus tip back L',
          'tarsus tip front R', 'tarsus tip mid R', 'tarsus tip back R']

crop_positions = ['/bottom_view/videos/crop_loc_191125_PR_Fly1_001_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly1_002_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly1_003_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly1_004_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_001_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_002_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_003_prism.txt',
                  '/bottom_view/videos/crop_loc_191125_PR_Fly2_004_prism.txt']

videos_side = ['side_view/videos/video_191125_PR_Fly1_001_prism',
               'side_view/videos/video_191125_PR_Fly1_002_prism',
               'side_view/videos/video_191125_PR_Fly1_003_prism',
               'side_view/videos/video_191125_PR_Fly1_004_prism',
               'side_view/videos/video_191125_PR_Fly2_001_prism',
               'side_view/videos/video_191125_PR_Fly2_002_prism',
               'side_view/videos/video_191125_PR_Fly2_003_prism',
               'side_view/videos/video_191125_PR_Fly2_004_prism']

videos_bottom =  ['bottom_view/videos/video_191125_PR_Fly1_001_prism',
                  'bottom_view/videos/video_191125_PR_Fly1_002_prism',
                  'bottom_view/videos/video_191125_PR_Fly1_003_prism',
                  'bottom_view/videos/video_191125_PR_Fly1_004_prism',
                  'bottom_view/videos/video_191125_PR_Fly2_001_prism',
                  'bottom_view/videos/video_191125_PR_Fly2_002_prism',
                  'bottom_view/videos/video_191125_PR_Fly2_003_prism',
                  'bottom_view/videos/video_191125_PR_Fly2_004_prism']

assert len(videos_side)==len(videos_bottom), 'Number of video files must be the same from side and bottom!'

In [16]:
def read_crop_pos(file):
    f=open(file, "r")
    contents =f.readlines()
    pos = []
    for i in range(4,len(contents)):
        line = contents[i][:-1].split(' ')
        pos.append(int(line[1]))
        
    return pos    

Filter for high quality frames

In [22]:
index = []
side = pd.DataFrame()
bottom = pd.DataFrame()
for i in tqdm(range(len(videos_side))):
    _side = pd.read_hdf(root_folder + videos_side[i] + scorer_side + '.h5')
    _bottom = pd.read_hdf(root_folder + videos_bottom[i] + scorer_bottom + '.h5')
    import sys
    sys.exit()
    
    #drop scorer column label
    _side = _side.droplevel('scorer',axis=1) 
    _bottom = _bottom.droplevel('scorer',axis=1) 
    
    #add location of crop to get global coordinates
    crop_loc = read_crop_pos(root_folder + crop_positions[i])
    _side.loc[:,(slice(None),'x')] = _side.loc[:,(slice(None),'x')].add(crop_loc,axis=0)
    _bottom.loc[:,(slice(None),'x')] = _bottom.loc[:,(slice(None),'x')].add(crop_loc,axis=0)
    
    #split L and R (remove if we include flipping) and select for high likelihood frames
    side_L_lk = _side.loc[:,(joints[:3],'likelihood')]
    side_R_lk = _side.loc[:,(joints[3:],'likelihood')]
    bottom_lk = _bottom.loc[:,(joints,'likelihood')]
    
    mask = ( ((side_L_lk>th1).sum(1)==3) | ((side_R_lk>th1).sum(1)==3) ) & ((bottom_lk>th1).sum(1)==6)
    _side = _side[mask].dropna()
    _bottom = _bottom[mask].dropna()
    
    #sometimes DLC mixes up limbs so take only those frames there the x coordinate matches on side and bottom views
    diff_L = np.abs(_bottom.loc[:,(joints[:3],'x')].values - _side.loc[:,(joints[3:],'x')].values)
    diff_R = np.abs(_bottom.loc[:,(joints[3:],'x')].values - _side.loc[:,(joints[:3],'x')].values)
    mask = ((diff_L<th2).sum(1)==3) | ((diff_R<th2).sum(1)==3)
    _side = _side[mask].dropna()
    _bottom = _bottom[mask].dropna()

    assert _side.shape[0]==_bottom.shape[0], 'Number of rows must match in filtered data!'
    
    #collect index
    side = side.append(_side)
    bottom = bottom.append(_bottom)
    index.append(_side.index.values)
    
    name = crop_positions[i].split('/')[-1].split('_')[2:]
    _side.to_hdf(root_folder + 'prediction_filtered_' + name[2] + '_' +name[3] + '.h5', key='side', mode='w')
    _bottom.to_hdf(root_folder + 'prediction_filtered_' + name[2] + '_' +name[3] + '.h5', key='bottom')

  0%|          | 0/8 [00:00<?, ?it/s]


SystemExit: 

/home/gosztolai/anaconda3/envs/DLC_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [18]:
#save
side.to_hdf(root_folder + 'prediction_filtered.h5', key='side', mode='w')
bottom.to_hdf(root_folder + 'prediction_filtered.h5', key='bottom')
side = [] #this is just to free up memory
bottom = []

Make joint video

In [19]:
import subprocess
import cv2
    
def imgs_to_video(imgs, fps, out_path):
    '''Write video from a list of images'''
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') #‘F’, ‘M’, ‘P’, ‘4’
    out = cv2.VideoWriter(out_path, fourcc, fps, (imgs[0].shape[1], imgs[0].shape[0]))
                          #imgs[0].shape[:2])
    for img in imgs:
        out.write(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    out.release()
    
                          
def video_to_imgs(vid_path):
    '''Convert video to a list of images'''
    
    cap = cv2.VideoCapture(vid_path)         
    imgs = []            
    while True:
        flag, frame = cap.read()
        if flag:                            
            imgs.append(frame)       
        else:
            break        
    
    return imgs
                                          
    
def concat_vids(vid_name):   
     
    cmd = 'ffmpeg -f concat -safe 0 -i videos.txt -c copy ' + vid_name
    subprocess.call(cmd, shell=True)

    
def videos_to_concat(paths):
    '''
    Make a textfile with paths to videos to concatenate
    '''
    f = open('videos.txt','w+')
    for i in paths:
        f.write("file '" + i + "'\r\n")
        
    f.close()

In [20]:
print('create videos')
paths = []
for i in tqdm(range(len(videos_side))):       
    #extract frames
    frames = video_to_imgs(root_folder + videos_side[i] + scorer_side + '_labeled.mp4') #convert to frames
    frames = [frames[j] for j in index[i]] #prune frames
    paths.append(root_folder + 'side_' + str(i) + '.mp4') #save output video path for later
    imgs_to_video(frames, 50, paths[i]) #convert back to video
frames = [] #to free up memory 
    
print('concatenate videos')
videos_to_concat(paths)
concat_vids(root_folder + 'side.mp4')
    
print('create videos')    
paths = []
for i in tqdm(range(len(videos_bottom))):    
    #extract frames
    frames = video_to_imgs(root_folder + videos_bottom[i] + scorer_bottom + '_labeled.mp4') #convert to frames
    frames = [frames[j] for j in index[i]] #prune frames
    paths.append(root_folder + 'bottom_' + str(i) + '.mp4') #save output video path for later
    imgs_to_video(frames, 50, paths[i]) #convert back to video

frames = [] #to free up memory    
print('concatenate videos')
videos_to_concat(paths)
concat_vids(root_folder + 'bottom.mp4')    

print('stack videos')
ffmpeg_cmd = "ffmpeg -i " + root_folder + "side.mp4 -i " + root_folder + "bottom.mp4 -filter_complex vstack=inputs=2 " + root_folder + "stacked.mp4"
subprocess.call(ffmpeg_cmd, shell=True)

  0%|          | 0/8 [00:00<?, ?it/s]

create videos


100%|██████████| 8/8 [00:24<00:00,  3.03s/it]


concatenate videos


  0%|          | 0/8 [00:00<?, ?it/s]

create videos


100%|██████████| 8/8 [00:38<00:00,  4.79s/it]


concatenate videos
stack videos


0